In [ ]:
from snowflake.snowpark import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from dotenv import load_dotenv
from snowflake.snowpark.functions import avg,stddev,udf
from snowflake.snowpark.types import PandasSeries
from datetime import datetime

In [ ]:
load_dotenv()

In [ ]:
# session=Session.builder.configs(
#     SnowflakeLoginOptions("test_conn")
# )

session=Session.builder.getOrCreate()

In [ ]:
session

In [ ]:
df=session.table("snowflake_sample_data.tpch_sf100.customer")
print(f"Count: {df.count()}")

In [ ]:
df_mean=float(
    df.agg(
        avg("C_ACCTBAL")
    ).collect()[0][0]
)
print(f"mean:{df_mean}")

In [ ]:
df_stddev=float(
    df.agg(
        stddev("C_ACCTBAL")
    ).collect()[0][0]
)
print(f"stddev:{df_stddev}")

In [ ]:
@udf()
def udf_mean_stddev(inp:float) -> float:
    return (inp-df_mean+df_stddev)*10000.0


@udf()
def udf_mean_stddev_vect(inp:PandasSeries[float]) -> PandasSeries[float]:
    return (inp-df_mean+df_stddev)*10000.0

In [ ]:
st=datetime.now()
df.select(
    udf_mean_stddev("C_ACCTBAL").alias("bal_from_mean")).\
        agg(avg('bal_from_mean')
    ).collect()
end=datetime.now()
print(f"udf_mean_stddev : {(end-st).total_seconds()} seconds")

In [ ]:
st=datetime.now()
df.select(
    udf_mean_stddev_vect("C_ACCTBAL").alias("bal_from_mean")).\
        agg(avg('bal_from_mean')
    ).collect()
end=datetime.now()
print(f"udf_mean_stddev_vect : {(end-st).total_seconds()} seconds")